## Deploy Final Detector

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import shutil
import os 
import glob
import csv
import json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix as confusion_matrix_sklearn

from ketos.data_handling import selection_table as sl
import ketos.data_handling.database_interface as dbi
from ketos.data_handling.parsing import load_audio_representation
from ketos.data_handling.data_feeding import BatchGenerator
from ketos.neural_networks.resnet import ResNetInterface
from ketos.audio.audio_loader import AudioFrameLoader, AudioLoader, SelectionTableIterator
from ketos.audio.spectrogram import MagSpectrogram
from ketos.neural_networks.dev_utils.detection import batch_load_audio_file_data, filter_by_threshold, filter_by_label, merge_overlapping_detections
from ketos.data_handling.data_feeding import JointBatchGen

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print('done importing packages')

done importing packages


C:\Users\kzammit\Miniconda3\envs\ketos_env\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Run on all audio files

In [3]:
main_folder = r'E:\baseline-w-normalization'
temp_folder_pp = main_folder + '\\' + 'ringedS_tmp_folder'
detections_csv = main_folder + '\\' + 'detections_raw.csv'
spectro_file = r'E:\baseline-w-normalization\spec_config_100-1200Hz-0.032-hamm-adjustrange.json'
model_name = r'E:\baseline-w-normalization\rs-model-4.kt'
audio_folder = r'E:\final-baseline-detector\audio'

# Look at detections above this threshold
threshold = 0.5

# Step 0.5s each time (overlap of 50% for 1 sec duration)
step_size = 0.5

# Number of samples in batch
batch_size = 16

model = ResNetInterface.load(model_file=model_name, new_model_folder=temp_folder_pp)

audio_repr = load_audio_representation(path=spectro_file)

spec_config = audio_repr['spectrogram']

audio_loader = AudioFrameLoader(path=audio_folder, duration=spec_config['duration'],
                                    step=step_size, stop=False, representation=spec_config['type'],
                                    representation_params=spec_config, pad=False)
detections = pd.DataFrame()

batch_generator = batch_load_audio_file_data(loader=audio_loader, batch_size=batch_size)

for batch_data in batch_generator:
    # Run the model on the spectrogram data from the current batch
    batch_predictions = model.run_on_batch(batch_data['data'], return_raw_output=True)

    # Lets store our data in a dictionary
    raw_output = {'filename': batch_data['filename'], 'start': batch_data['start'], 'end': batch_data['end'],
                  'score': batch_predictions}

    batch_detections = filter_by_threshold(raw_output, threshold=threshold)

    detections = pd.concat([detections, batch_detections], ignore_index=True)

detections.to_csv(detections_csv, index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 8406/8406 [14:54<00:00,  9.40it/s]


In [4]:
# Filter the detections for only the positive results 
detections_filtered = filter_by_label(detections, labels=1).reset_index(drop=True)
print(detections_filtered)

                                      filename  start    end  label     score
0                  1208795168.700911064318.wav    0.5    1.5      1  0.999533
1                  1208795168.700911064318.wav    1.0    2.0      1  0.998006
2                  1208795168.700911064318.wav    1.5    2.5      1  0.999894
3                  1208795168.700911064318.wav    2.0    3.0      1  0.995934
4                  1208795168.700911064318.wav    2.5    3.5      1  0.983766
...                                        ...    ...    ...    ...       ...
14046  test\2017-ULU01_0+1_20180323_090602.wav  273.5  274.5      1  0.994432
14047  test\2017-ULU01_0+1_20180323_090602.wav  274.0  275.0      1  0.996855
14048  test\2017-ULU01_0+1_20180323_090602.wav  291.0  292.0      1  0.702005
14049  test\2017-ULU01_0+1_20180323_090602.wav  291.5  292.5      1  0.916594
14050  test\2017-ULU01_0+1_20180323_090602.wav  296.5  297.5      1  0.528207

[14051 rows x 5 columns]


In [6]:
# Merge overlapping detections 
#The score of the merged detection is computed as the average of the individual detection scores.

detections_grp = merge_overlapping_detections(detections_filtered)
print(detections_grp)
detections_grp.to_csv(r'E:\baseline-w-normalization\pos_detections_all_audio.csv', index=False)

                                     filename  start    end  label     score
0                 1208795168.700911064318.wav    0.5    4.0      1  0.908550
1                 1208795168.700911064318.wav    9.0   12.0      1  0.964775
2                 1208795168.700911064318.wav   15.5   20.5      1  0.696002
3                 1208795168.700911064318.wav   23.5   25.0      1  0.830665
4                 1208795168.700911064318.wav   33.0   34.5      1  0.815544
...                                       ...    ...    ...    ...       ...
4883  test\2017-ULU01_0+1_20180323_090602.wav  260.0  261.0      1  0.769013
4884  test\2017-ULU01_0+1_20180323_090602.wav  271.0  273.0      1  0.822779
4885  test\2017-ULU01_0+1_20180323_090602.wav  273.5  275.0      1  0.995644
4886  test\2017-ULU01_0+1_20180323_090602.wav  291.0  292.5      1  0.809300
4887  test\2017-ULU01_0+1_20180323_090602.wav  296.5  297.5      1  0.528207

[4888 rows x 5 columns]


In [10]:
#Define a function that compares the upcalls found by the model (detections)
#with the upcalls identified by the human expert (annotations).
#The function returns the annotation DataFrame with an extra boolean 
#column indicating if a given annotated upcall was detected by the model.
def compare(annotations, detections):

    detected_list = []
    detections['annotation'] = 'NO'

    for idx,row in annotations.iterrows(): #loop over annotations
        filename_annot = row['filename']
        time_annot = row['start'] + row['end'] /2
        detected = False
        for _, d in detections.iterrows(): #loop over detections
            filename_det = d['filename']
            start_det    = d['start']
            end_det      = start_det + d['end']
            # if the filenames match and the annotated time falls with the start and 
            # end time of the detection interval, consider the call detected
            if filename_annot==filename_det and time_annot >= start_det and time_annot <= end_det:
                detections.at[_, 'annotation'] = 'YES'
                detected = True
                break

        detected_list.append(detected)       

    annotations['detected'] = detected_list  #add column to the annotations table
    detections.to_csv(r'E:\final-baseline-detector\pearce-point\detections-w-annot-flag.csv', index=False)
    
    return annotations

#call the function
annots = pd.read_csv(r'E:\final-baseline-detector\pearce-point\PP_2018.2019_AllYear.txt', delimiter='\t')
annotation = compare(annots, detections_grp)

In [8]:
results_table = pd.read_csv(r'E:\baseline-w-normalization\pos_detections_all_audio.csv')
audio_folder = r'E:\final-baseline-detector\audio'

cols = ['filename']
results_table.loc[:,cols] = results_table.loc[:,cols].ffill()
results_table['Selection'] = results_table.index +1
results_table['View'] = 'Spectrogram 1'
results_table['Channel'] = 1
results_table['Begin Path'] = audio_folder + '\\' + results_table.filename
results_table['File Offset (s)'] = results_table.start
results_table = results_table.rename(columns={"start": "Begin Time (s)", "end": "End Time (s)", "filename": "Begin File"})
results_table['Begin File'] = results_table['Begin File']
results_table['Low Freq (Hz)'] = 100
results_table['High Freq (Hz)'] = 1200

results_table.to_csv(r'E:\baseline-w-normalization\raven-formatted-detections-model4.txt',
                     index=False, sep='\t')